<a href="https://colab.research.google.com/github/JoaoVitorSantiagoNogueira/case-brick-seguros/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extração
Aqui nós extraimos os dados da internet dos carros

In [21]:
import requests
import re
import pandas as pd

## Extrair todos os cógidos FIPE

Feito manualmente, todos os códigos FIPE estão presentes na pagina:
https://www.tabelafipebrasil.com/fipe/carros
bastou fazer um regex no formato [\d]+-[\d]


arquivo gerado ficou diisponível em "https://github.com/JoaoVitorSantiagoNogueira/case-brick-seguros/blob/main/FIPE-CODES.txt"

In [27]:
def get_fipe_codes():
  codes_string = requests.get("https://raw.githubusercontent.com/JoaoVitorSantiagoNogueira/case-brick-seguros/main/FIPE-CODES.txt").text
  codes_list = re.split("\r\n",codes_string)
  return codes_list

In [28]:
len(get_fipe_codes())

6893

## Pegar Informações de Veiculo

In [29]:
# https://brasilapi.com.br/api/fipe/preco/v1/038003-2  Últimos números são o código fipe

def get_json_from_fipe(Codigo_FIPE, query = ""):

  if query != "":
    query = "?tabela_referencia="+query

  response = requests.get("https://brasilapi.com.br/api/fipe/preco/v1/" + Codigo_FIPE + query)
  return response.json()


In [26]:
#sample
type(get_json_from_fipe("038003-2", "9"))

list

#Cria a tabela inteira de um veiculo
Navegando preenchendo mês-a-mês, no formato mm-aa

## Cria o data frame base

In [6]:
df = pd.json_normalize(get_json_from_fipe("038003-2", "10"))
df

,valor,marca,modelo,anoModelo,combustivel,codigoFipe,mesReferencia,tipoVeiculo,siglaCombustivel,dataConsulta
0,"R$ 18.742,00",Acura,Integra GS 1.8,1992,Gasolina,038003-2,março de 2006,1,G,"segunda-feira, 13 de maio de 2024 15:02"
1,"R$ 17.197,00",Acura,Integra GS 1.8,1991,Gasolina,038003-2,março de 2006,1,G,"segunda-feira, 13 de maio de 2024 15:02"


from matplotlib import pyplot as plt
df['anoModelo'].plot(kind='hist', bins=20, title='anoModelo')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
df.groupby('valor').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['tipoVeiculo']
  ys = series['anoModelo']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = df.sort_values('tipoVeiculo', ascending=True)
for i, (series_name, series) in enumerate(df_sorted.groupby('valor')):
  _plot_series(series, series_name, i)
  fig.legend(title='valor', bbox_to_anchor=(1, 1), loc='upper left')
sns.despine(fig=fig, ax=ax)
plt.xlabel('tipoVeiculo')
_ = plt.ylabel('anoModelo')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['tipoVeiculo']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'tipoVeiculo'}, axis=1)
              .sort_values('tipoVeiculo', ascending=True))
  xs = counted['tipoVeiculo']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = df.sort_values('tipoVeiculo', ascending=True)
for i, (series_name, series) in enumerate(df_sorted.groupby('valor')):
  _plot_series(series, series_name, i)
  fig.legend(title='valor', bbox_to_anchor=(1, 1), loc='upper left')
sns.despine(fig=fig, ax=ax)
plt.xlabel('tipoVeiculo')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
df['anoModelo'].plot(kind='line', figsize=(8, 4), title='anoModelo')
plt.gca().spines[['top', 'right']].set_visible(False)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(df['valor'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(df, x='anoModelo', y='valor', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [78]:
#processo demorado, em torno de 1 minuto por veiculo
def create_whole_dataset(codigos):
  #criar um dataframe e atualizar eles é custoso
  #https://stackoverflow.com/questions/13784192/creating-an-empty-pandas-dataframe-and-then-filling-it
  #fazemos primeiro em dicts e depois criamos o data frame

#paralelização com código baseado em:
# https://stackoverflow.com/questions/57126286/fastest-parallel-requests-in-python



  data_set = []

  for codigo in codigos: ## cada linha gera sua pesquisa própria
    new_line = get_json_from_fipe(codigo)
    #completar com outros anos de dados da tabela fipe

    i=0
    entrada_atual = new_line[0]["mesReferencia"]
    ultima_entrada = None
    while (ultima_entrada != entrada_atual):
      i += 1
      serie_temporal = get_json_from_fipe(codigo, str(i))
      if type(serie_temporal) == list :
        ultima_entrada = serie_temporal[0]["mesReferencia"]
        for modelo in range(len(new_line)):
          new_line[modelo][ultima_entrada] = serie_temporal[modelo]["valor"]
    data_set.append(new_line)

  return data_set

In [80]:
result = create_whole_dataset(["038003-2"])
print(result)

[[{'valor': 'R$ 11.417,00', 'marca': 'Acura', 'modelo': 'Integra GS 1.8', 'anoModelo': 1992, 'combustivel': 'Gasolina', 'codigoFipe': '038003-2', 'mesReferencia': 'maio de 2024 ', 'tipoVeiculo': 1, 'siglaCombustivel': 'G', 'dataConsulta': 'segunda-feira, 13 de maio de 2024 15:14', 'setembro de 2005 ': 'R$ 18.445,00', 'outubro de 2005 ': 'R$ 18.421,00', 'novembro de 2005 ': 'R$ 18.522,00', 'agosto de 2005 ': 'R$ 18.378,00', 'janeiro de 2006 ': 'R$ 18.585,00', 'fevereiro de 2006 ': 'R$ 18.679,00', 'março de 2006 ': 'R$ 18.742,00', 'abril de 2006 ': 'R$ 18.770,00', 'maio de 2006 ': 'R$ 18.975,00', 'junho de 2006 ': 'R$ 18.786,00', 'julho de 2006 ': 'R$ 18.784,00', 'agosto de 2006 ': 'R$ 18.791,00', 'setembro de 2006 ': 'R$ 18.763,00', 'outubro de 2006 ': 'R$ 18.812,00', 'novembro de 2006 ': 'R$ 18.320,00', 'dezembro de 2006 ': 'R$ 18.747,00', 'janeiro de 2007 ': 'R$ 18.679,00', 'fevereiro de 2007 ': 'R$ 18.824,00', 'março de 2007 ': 'R$ 18.738,00', 'abril de 2007 ': 'R$ 18.669,00', 'julho